In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
train_models(env, models, episodes=10, steps=2000, print_every=500)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: 0.9799999999999898
[6] [VIRUS] Agent0 ate virus 6
----STEP 500 rewards----
Model 0: 124.72059036504106
----STEP 1000 rewards----
Model 0: 111.79278392809496
----STEP 1500 rewards----
Model 0: 105.4205912258501
------EPISODE 0 rewards------
Model 0: 102.72357886571028
----STEP 0 rewards----
Model 0: 0.9799999999999898
----STEP 500 rewards----
Model 0: 0.7426862255321964
----STEP 1000 rewards----
Model 0: -9.048127521064117
----STEP 1500 rewards----
Model 0: -18.36141436183408
------EPISODE 1 rewards------
Model 0: -27.20318501296083
----STEP 0 rewards----
Model 0: 0.9799999999999898
[9] [VIRUS] Agent2 ate virus 19
----LEARNING BEGINS----
----STEP 500 rewards----
Model 0: 105.3888309562802
----STEP 1000 rewards----
Model 0: 115.43785823942285
----STEP 1500 rewards----
Model 0: 120.57873338892519
------EPISODE 2 rewards------
Model 0: 111.57186824731869
----STEP 0 rewards----
Model 0: 1.9799999999999898
----STEP 500 rewards----
Model 0: -4.01462

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_12")